In [2]:
pip install netcal

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.4/47.4 kB 403.5 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 1.5 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 1.6 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.3/236.3 kB 1.9 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.3/53.3 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.1/274.1 kB 3.6 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.3/7.3 MB 3.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.0/14.0 MB 3.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 756.0/756.0 kB 3.9 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 3.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.4/78.4 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.7

In [1]:
import h5py
import pandas as pd
import numpy as np
import cv2
import os
import matplotlib.pyplot as plt
from collections import Counter
from sklearn.model_selection import train_test_split
import pickle
import time
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import argparse
import json
import hashlib
from sklearn.metrics import f1_score, precision_score, recall_score, roc_curve, confusion_matrix
from netcal.metrics import ECE

import sys

if 'ipykernel_launcher' in sys.argv[0]:
    sys.argv = sys.argv[:1]  # حذف آرگومان‌های ناخواسته

# Define paths and read dataset
path = '/Users/amir/PycharmProjects/Medfair/MEDFAIR/'
demo_data = pd.read_excel(path + 'BrEaST-Lesions-USG-clinical-data-Dec-15-2023.xlsx')
images_path = os.path.join(path, '/MEDFAIR/BrEaST-Lesions_USG-images_and_masks/')
pathlist = demo_data['Image_filename'].values.tolist()
paths = ['/Users/amir/PycharmProjects/Medfair/MEDFAIR/BrEaST-Lesions_USG-images_and_masks/' + i for i in pathlist]
demo_data['Path'] = paths

# Data preprocessing
demo_data = demo_data[~demo_data['Age'].isnull()]
age_bins = [0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100]
age_labels = ['0-10', '11-20', '21-30', '31-40', '41-50', '51-60', '61-70', '71-80', '81-90', '91-100']
demo_data['Age_Category'] = pd.cut(demo_data['Age'], bins=age_bins, labels=age_labels, right=False)

demo_data['Age_multi'] = demo_data['Age'].values.astype('int')
demo_data['Age_multi'] = np.where(demo_data['Age_multi'].between(-1, 19), 0, demo_data['Age_multi'])
demo_data['Age_multi'] = np.where(demo_data['Age_multi'].between(20, 39), 1, demo_data['Age_multi'])
demo_data['Age_multi'] = np.where(demo_data['Age_multi'].between(40, 59), 2, demo_data['Age_multi'])
demo_data['Age_multi'] = np.where(demo_data['Age_multi'].between(60, 79), 3, demo_data['Age_multi'])
demo_data['Age_multi'] = np.where(demo_data['Age_multi'] >= 80, 4, demo_data['Age_multi'])

demo_data['Age_binary'] = demo_data['Age'].values.astype('int')
demo_data['Age_binary'] = np.where(demo_data['Age_binary'].between(-1, 60), 0, demo_data['Age_binary'])
demo_data['Age_binary'] = np.where(demo_data['Age_binary'] >= 60, 1, demo_data['Age_binary'])

labels = demo_data['Classification'].values.copy()
labels[labels == 'malignant'] = '1'
labels[labels != '1'] = '0'
labels = labels.astype('int')
demo_data['binaryLabel'] = labels

def split_811(all_meta, patient_ids):
    sub_train, sub_val_test = train_test_split(patient_ids, test_size=0.2, random_state=0)
    sub_val, sub_test = train_test_split(sub_val_test, test_size=0.5, random_state=0)
    train_meta = all_meta[all_meta.CaseID.isin(sub_train)]
    val_meta = all_meta[all_meta.CaseID.isin(sub_val)]
    test_meta = all_meta[all_meta.CaseID.isin(sub_test)]
    return train_meta, val_meta, test_meta

sub_train, sub_val, sub_test = split_811(demo_data, np.unique(demo_data['CaseID']))
sub_train.to_csv('/Users/amir/PycharmProjects/Medfair/MEDFAIR/split/new_train.csv')
sub_val.to_csv('/Users/amir/PycharmProjects/Medfair/MEDFAIR/split/new_val.csv')
sub_test.to_csv('/Users/amir/PycharmProjects/Medfair/MEDFAIR/split/new_test.csv')

class CustomDataset(Dataset):
    def __init__(self, dataframe, transform=None):
        self.dataframe = dataframe
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        img_path = self.dataframe.iloc[idx]['Path']
        image = Image.open(img_path).convert('RGB')
        label = self.dataframe.iloc[idx]['binaryLabel']
        protected_attr = self.dataframe.iloc[idx]['Age_Category']

        if self.transform:
            image = self.transform(image)

        return image, label, protected_attr

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

train_dataset = CustomDataset(dataframe=sub_train, transform=transform)
val_dataset = CustomDataset(dataframe=sub_val, transform=transform)
test_dataset = CustomDataset(dataframe=sub_test, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Define the models
model_resnet18 = models.resnet18(pretrained=True)
num_ftrs = model_resnet18.fc.in_features
model_resnet18.fc = nn.Linear(num_ftrs, 2)

model_vgg16 = models.vgg16(pretrained=True)
num_ftrs = model_vgg16.classifier[6].in_features
model_vgg16.classifier[6] = nn.Linear(num_ftrs, 2)

model_densenet = models.densenet121(pretrained=True)
num_ftrs = model_densenet.classifier.in_features
model_densenet.classifier = nn.Linear(num_ftrs, 2)

model_mobilenet = models.mobilenet_v2(pretrained=True)
num_ftrs = model_mobilenet.classifier[1].in_features
model_mobilenet.classifier[1] = nn.Linear(num_ftrs, 2)

model_alexnet = models.alexnet(pretrained=True)
num_ftrs = model_alexnet.classifier[6].in_features
model_alexnet.classifier[6] = nn.Linear(num_ftrs, 2)

# Criterion and optimizers
criterion = nn.CrossEntropyLoss()
optimizers = [
    optim.SGD(model_resnet18.parameters(), lr=0.001, momentum=0.9),
    optim.SGD(model_vgg16.parameters(), lr=0.001, momentum=0.9)
]

optimizers = [
    optim.SGD(model_resnet18.parameters(), lr=0.001, momentum=0.9)
]

def collect_args():
    parser = argparse.ArgumentParser()
    parser.add_argument('--experiment', type=str, choices=['baseline', 'CFair', 'LAFTR', 'resampling'])
    parser.add_argument('--sensitive_name', default='Age_Category', choices=['Age_Category'])
    parser.add_argument('--random_seed', type=int, default=0)
    parser.add_argument('--batch_size', type=int, default=32)
    parser.add_argument('--lr', type=float, default=1e-4)
    parser.add_argument('--weight_decay', type=float, default=1e-4)
    parser.add_argument('--total_epochs', type=int, default=5)
    parser.add_argument('--fair_coeff', type=float, default=1.0)
    parser.set_defaults(cuda=False)  # Default to False since CUDA is not available
    
    opt = vars(parser.parse_args())
    opt = create_experiment_setting(opt)
    return opt

def create_experiment_setting(opt):
    run_hash = hashlib.sha1()
    run_hash.update(str(time.time()).encode('utf-8'))
    opt['hash'] = run_hash.hexdigest()[:10]
    print('run hash (first 10 digits): ', opt['hash'])
    
    opt['device'] = torch.device('cuda' if opt['cuda'] and torch.cuda.is_available() else 'cpu')
    
    return opt

opt = collect_args()

def train_model(model, dataloaders, criterion, optimizer, opt, num_epochs= 5 ):
    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)

        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0

            for inputs, labels, protected_attrs in dataloaders[phase]:
                inputs = inputs.to(opt['device'])
                labels = labels.to(opt['device'])

                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    if phase == 'train':
                        if opt['experiment'] == 'CFair':
                            fair_loss = compute_fairness_loss(outputs, protected_attrs, opt['fair_coeff'])
                            loss += fair_loss
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

        print()

    return model

# Train the models
device = torch.device("cpu")  # Use CPU as CUDA is not available

models_list = [model_resnet18, model_vgg16, model_densenet, model_mobilenet, model_alexnet]
optimizers = [optim.SGD(model.parameters(), lr=0.001, momentum=0.9) for model in models_list]
model_names = ['ResNet18', 'VGG16', 'DenseNet121', 'MobileNetV2', 'AlexNet']

dataloaders = {
    'train': train_loader,
    'val': val_loader
}

trained_models = []

for model, optimizer, model_name in zip(models_list, optimizers, model_names):
    model = model.to(device)
    print(f"Training {model_name}...")
    trained_model = train_model(model, dataloaders, criterion, optimizer, opt, num_epochs=5)
    trained_models.append(trained_model)
    torch.save(trained_model.state_dict(), f'model_{model_name}.pth')
    print(f"{model_name} trained and saved.")

# Define evaluation functions
def calculate_fairness_metrics(y_true, y_pred, sensitive_attr):
    cm = confusion_matrix(y_true, y_pred)
    tn, fp, fn, tp = cm.ravel()

    FPR = fp / (fp + tn)
    FNR = fn / (fn + tp)
    TPR = tp / (tp + fn)
    TNR = tn / (tn + fp)
    PR80_TNR = TPR / TNR if TNR != 0 else np.inf
    
    BCE = (FPR + FNR) / 2

    ece = ECE().measure(y_pred, y_true)
    
    # Balanced Equalized Odds (BEO) calculation
    groups = np.unique(sensitive_attr)
    TPR_diff = []
    FPR_diff = []
    for group in groups:
        group_idx = (sensitive_attr == group)
        group_y_true = y_true[group_idx]
        group_y_pred = y_pred[group_idx]
        cm_group = confusion_matrix(group_y_true, group_y_pred)
        tn_g, fp_g, fn_g, tp_g = cm_group.ravel()

        FPR_g = fp_g / (fp_g + tn_g) if (fp_g + tn_g) > 0 else 0
        TPR_g = tp_g / (tp_g + fn_g) if (tp_g + fn_g) > 0 else 0

        FPR_diff.append(FPR_g)
        TPR_diff.append(TPR_g)
    
    FPR_diff = np.abs(np.diff(FPR_diff)).sum() / (len(groups) - 1)
    TPR_diff = np.abs(np.diff(TPR_diff)).sum() / (len(groups) - 1)

    Balanced_EO = (FPR_diff + TPR_diff) / 2
    
    metrics = {
        "BCE": BCE,
        "ECE": ece,
        "FPR": FPR,
        "FNR": FNR,
        "PR80_TNR": PR80_TNR,
        "Balanced_EO": Balanced_EO
    }
    
    return metrics


/Users/amir/Documents/Projects/Python/Medfair/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/amir/Documents/Projects/Python/Medfair/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/Users/amir/Documents/Projects/Python/Medfair/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.

run hash (first 10 digits):  338e8abe92
Training ResNet18...
Epoch 0/4
----------
train Loss: 0.7456 Acc: 0.4912
val Loss: 0.7178 Acc: 0.5238

Epoch 1/4
----------
train Loss: 0.6086 Acc: 0.6725
val Loss: 0.6920 Acc: 0.6190

Epoch 2/4
----------
train Loss: 0.4619 Acc: 0.8655
val Loss: 0.6344 Acc: 0.7143

Epoch 3/4
----------
train Loss: 0.3724 Acc: 0.8713
val Loss: 0.5941 Acc: 0.7143

Epoch 4/4
----------
train Loss: 0.2698 Acc: 0.9532
val Loss: 0.5723 Acc: 0.7619

ResNet18 trained and saved.
Training VGG16...
Epoch 0/4
----------
train Loss: 0.6831 Acc: 0.5556
val Loss: 0.7003 Acc: 0.5714

Epoch 1/4
----------
train Loss: 0.6392 Acc: 0.5965
val Loss: 0.6378 Acc: 0.6190

Epoch 2/4
----------
train Loss: 0.6070 Acc: 0.7018
val Loss: 0.6231 Acc: 0.5238

Epoch 3/4
----------
train Loss: 0.5345 Acc: 0.7076
val Loss: 0.5645 Acc: 0.7619

Epoch 4/4
----------
train Loss: 0.4980 Acc: 0.7544
val Loss: 0.9386 Acc: 0.5714

VGG16 trained and saved.
Training DenseNet121...
Epoch 0/4
----------
tra

In [2]:
from sklearn.metrics import balanced_accuracy_score, roc_auc_score, confusion_matrix


def expected_calibration_error(y_true, y_pred):
    y_pred = y_pred.astype(np.float32)
    ece = ECE()
    return ece.measure(y_pred, y_true)

def false_positive_rate(y_true, y_pred):
    cm = confusion_matrix(y_true, y_pred)
    tn, fp, _, _ = cm.ravel()
    return fp / (fp + tn)

def false_negative_rate(y_true, y_pred):
    cm = confusion_matrix(y_true, y_pred)
    _, _, fn, tp = cm.ravel()
    return fn / (fn + tp)

def pr_at_tnr(y_true, y_pred, threshold=0.8):
    fpr, tpr, thresholds = roc_curve(y_true, y_pred)
    idx = np.argmax(fpr >= threshold)
    return tpr[idx]

def balanced_equalized_odds(y_true, y_pred, protected_attrs):
    groups = np.unique(protected_attrs)
    fpr_diff = []
    fnr_diff = []
    
    for group in groups:
        group_idx = (protected_attrs == group)
        group_y_true = y_true[group_idx]
        group_y_pred = y_pred[group_idx]
        
        fpr_diff.append(false_positive_rate(group_y_true, group_y_pred))
        fnr_diff.append(false_negative_rate(group_y_true, group_y_pred))
    
    fpr_diff = np.abs(np.diff(fpr_diff)).sum() / (len(groups) - 1)
    fnr_diff = np.abs(np.diff(fnr_diff)).sum() / (len(groups) - 1)
    
    return (fpr_diff + fnr_diff) / 2

In [3]:
# ایجاد یک نگاشت از دسته‌بندی‌ها به اعداد
age_category_mapping = {
    '0-10': 0,
    '11-20': 1,
    '21-30': 2,
    '31-40': 3,
    '41-50': 4,
    '51-60': 5,
    '61-70': 6,
    '71-80': 7,
    '81-90': 8,
    '91-100': 9
}

def evaluate_model(model, test_loader, opt):
    model.eval()
    all_labels = []
    all_preds = []
    all_protected_attrs = []

    with torch.no_grad():
        for inputs, labels, protected_attrs in test_loader:
            inputs = inputs.to(opt['device'])
            labels = labels.to(opt['device'])

            # نگاشت protected_attrs به کدهای عددی با استفاده از age_category_mapping
            if isinstance(protected_attrs, str):
                protected_attrs = torch.tensor([age_category_mapping[protected_attrs]]).to(opt['device'])
            else:
                # اگر protected_attrs یک آرایه یا لیست از دسته‌بندی‌ها بود
                protected_attrs = torch.tensor([age_category_mapping[attr] for attr in protected_attrs]).to(opt['device'])

            preds = model(inputs)

            all_labels.extend(labels.cpu().numpy())
            all_preds.extend(preds.argmax(dim=1).cpu().numpy())  # تبدیل احتمالات به برچسب‌های کلاس
            all_protected_attrs.extend(protected_attrs.cpu().numpy())

    all_labels = np.array(all_labels)
    all_preds = np.array(all_preds)
    all_protected_attrs = np.array(all_protected_attrs)

    fairness_metrics = {
        'BCE': balanced_accuracy_score(all_labels, all_preds),
        'ECE': expected_calibration_error(all_labels, all_preds),
        'FPR': false_positive_rate(all_labels, all_preds),
        'FNR': false_negative_rate(all_labels, all_preds),
        'PR @ 80 TNR': pr_at_tnr(all_labels, all_preds, threshold=0.8),
        'Balanced Equalized Odds': balanced_equalized_odds(all_labels, all_preds, all_protected_attrs),
    }

    return fairness_metrics

In [4]:
# ارزیابی مدل‌ها
print("Evaluating models on the test set...")
for model_name, model in zip(model_names, trained_models):
    fairness_metrics = evaluate_model(model, test_loader, opt)
    print(f"Fairness metrics for {model_name}: {fairness_metrics}")

# نتایج باید اکنون شامل BCE، ECE، FPR، FNR، PR @ 80 TNR و Balanced Equalized Odd

Evaluating models on the test set...
Fairness metrics for ResNet18: {'BCE': 0.811965811965812, 'ECE': 0.18181806260889227, 'FPR': 0.15384615384615385, 'FNR': 0.2222222222222222, 'PR @ 80 TNR': 1.0, 'Balanced Equalized Odds': 0.375}
Fairness metrics for VGG16: {'BCE': 0.5, 'ECE': 0.40909078988161957, 'FPR': 0.0, 'FNR': 1.0, 'PR @ 80 TNR': 1.0, 'Balanced Equalized Odds': 0.0}
Fairness metrics for DenseNet121: {'BCE': 0.5, 'ECE': 0.40909078988161957, 'FPR': 0.0, 'FNR': 1.0, 'PR @ 80 TNR': 1.0, 'Balanced Equalized Odds': 0.0}
Fairness metrics for MobileNetV2: {'BCE': 0.7564102564102564, 'ECE': 0.2272726080634377, 'FPR': 0.15384615384615385, 'FNR': 0.3333333333333333, 'PR @ 80 TNR': 1.0, 'Balanced Equalized Odds': 0.6875}
Fairness metrics for AlexNet: {'BCE': 0.6452991452991452, 'ECE': 0.3181816989725287, 'FPR': 0.15384615384615385, 'FNR': 0.5555555555555556, 'PR @ 80 TNR': 1.0, 'Balanced Equalized Odds': 0.3541666666666667}


In [5]:
import pandas as pd

# تبدیل دیکشنری به DataFrame
fairness_df = pd.DataFrame([fairness_metrics], index=[model_name])

# چاپ نتایج به صورت جدول
print(fairness_df)

              BCE       ECE       FPR       FNR  PR @ 80 TNR  \
AlexNet  0.645299  0.318182  0.153846  0.555556          1.0   

         Balanced Equalized Odds  
AlexNet                 0.354167  


In [7]:
import json

# چاپ نتایج به صورت مرتب شده
print(json.dumps(fairness_metrics, indent=4))

{
    "BCE": 0.6452991452991452,
    "ECE": 0.3181816989725287,
    "FPR": 0.15384615384615385,
    "FNR": 0.5555555555555556,
    "PR @ 80 TNR": 1.0,
    "Balanced Equalized Odds": 0.3541666666666667
}


In [9]:
print(model_name)

for metric, value in sorted(fairness_metrics.items()):
    print(f"{metric}: {value}")

AlexNet
BCE: 0.6452991452991452
Balanced Equalized Odds: 0.3541666666666667
ECE: 0.3181816989725287
FNR: 0.5555555555555556
FPR: 0.15384615384615385
PR @ 80 TNR: 1.0
